In [0]:
"""
CSCI 680-0K11 - Assignment 1 - Fall 2019

Author: Sneha Ravi Chandran
Z-ID: Z1856678
Date Due: September 20, 2019

Purpose: This program is to write a python code that 
         uses Naive Bayes algorithm to classify the 
         provided images.
"""
import numpy as np
import pandas as pd
import math
from urllib.request import *

# Opens training labels present in the URL and reads each line of the file into a list
with urlopen("https://raw.githubusercontent.com/sneharavi/CSCI-680-0K11-Neural-Networks/master/traininglabels") as trainingLabels:
  
  lines = trainingLabels.read().decode('utf-8').replace('\n', '')
  tLabels = []                                                                  
  for i in range(len(lines)):                                                    
    tLabels.append(lines[i])  
    
  #tlabel is a list that holds individual values present from traininglabels 
  tLabels = [int(i) for i in tLabels] 

  
"""
declaring an empty dictionary freq and updating the dictionary with the keys holding 
class value [0 to 9] and values holding the occurence of each of those number from the 
training labels.
"""
freq = {} 
for item in tLabels: 
  if (item in freq): 
    freq[item] += 1
  else: 
    freq[item] = 1         
 
"""
Declare a 3D list imageMartix which is later used to calculate the Probability 
for each of the 10 class values across a 28x28 and also passing the laplace Smoothing as 0.5
"""
imageMatrix = [[[0 for i in range(0,28)] for j in range( 0,28 ) ] for k in range( 0,10 )]

for i in freq.keys():
  for m in range(0,28):
    for n in range(0,28):
      imageMatrix[i][m][n] = (0.5 , freq[i] , freq[i]/5000)      
#print(imageMatrix)

"""
Opens the trainingimage present in the URL and reads each line of the file into a list
List arrTraining holds 28x28 for each of the 5000 values 
"""
arrTraining = []
with urlopen("https://raw.githubusercontent.com/sneharavi/CSCI-680-0K11-Neural-Networks/master/trainingimages") as trainingImages:
  for line in trainingImages:
    line = line.decode('utf-8').rstrip("\n\r")
    arrTraining.append(list(line))
arrTraining = np.array(arrTraining)
arrTraining = np.reshape(arrTraining,(5000,28,28))

#tImages holds the list of the values of trainingimage by converting the numpy array arrTraining to a list
tImages = arrTraining.tolist()

"""
used to calculate the feature values where it finds + or #, adds one to it which is 
the first tuple followed by the occurence of the class and prior value of it as second and third tuple value.
"""

for v in range(0,5000):
  for i in range(0,28):
    for j in range(0,28):
      if tImages[v][i][j] is not ' ':
        zeroField = (imageMatrix[int(tLabels[v])][i][j][0]+1)
        oneField = (imageMatrix[int(tLabels[v])][i][j][1])
        twoField = (imageMatrix[int(tLabels[v])][i][j][2])                                                                                                    

        imageMatrix[int(tLabels[v])][i][j] = (zeroField, oneField, twoField)        
        
"""
classification of the images from the file.
Opens the testlabels present in the URL and reads each line of the file into a list testLabelList
"""

with urlopen("https://raw.githubusercontent.com/sneharavi/CSCI-680-0K11-Neural-Networks/master/testlabels") as testLabels:
  testLabel = testLabels.read().decode('utf-8').replace('\n', '')
  testLabelList = []
  for i in range(len(testLabel)):
    testLabelList.append(testLabel[i])  
  testLabelList = [int(i) for i in testLabelList] 

"""
Opens the testimage present in the URL and reads each line of the file into a list
List arrTraining holds 28x28 for each of the 5000 values 
"""
arrTest = []
with urlopen("https://raw.githubusercontent.com/sneharavi/CSCI-680-0K11-Neural-Networks/master/testimages") as testingImages:
  for line in testingImages:
    line = line.decode('utf-8').rstrip("\n\r")
    arrTest.append(list(line))
arrTest = np.array(arrTest)
arrTest = np.reshape(arrTest,(1000,28,28))

#testImages holds the list of the values of testimages by converting the numpy array arrTest to a list
testImages = arrTest.tolist()



In [0]:
"""
for each of the 1000 values from the test labels, nb_val calculates the 
probability for each class from 0 to 9 across a 28*28 from the test image
the likelihood and compute using the log of each probability
"""
computedValue = []
for x in range(0,1000):
  nbv =[]
  for v in range(0,10):
    nb_val = 0
    for i in range(0,28):
      for j in range(0,28):
        if testImages[x][i][j] is not " ":
          zeroField = imageMatrix[v][i][j][0]
          oneField = imageMatrix[v][i][j][1]
          twoField = imageMatrix[v][i][j][2]
          nb_val = nb_val + math.log( zeroField / oneField * twoField )
      nb_val = nb_val + math.log(freq[v]/1000)
    nbv.append([v,nb_val])
       
  # nb_valMax is the list that hold the value after the calculation and find the
  # maximum value maxVal from the list
  nb_valMax= []
  for i in range(0,10):
    nb_valMax.append(nbv[i][1])
  maxVal = max(nb_valMax)
  
  for j in range(0,10):
    if maxVal == nbv[j][1]:
      computedValue.append(nbv[j])
#print(computedValue)
  


In [9]:
"""
import pandas to compute the confusion matrix by considering the actual given value 
against the predicted for the test labels and printing it.
"""

givenValue = pd.Series(testLabelList, name='Given')
confusionMatrix=[]
for i in range(0,1000):
  confusionMatrix.append(computedValue[i][0])
#print(confusionMatrix)

print("--------------- Confusion Matrix: ----------------\n")
predictedValue = pd.Series(confusionMatrix, name='Predicted')
df_confusionMatrix = pd.crosstab(givenValue, predictedValue)
#print(df_confusion)

# normalizing the confusion matrix
df_CMNormalized = df_confusionMatrix / df_confusionMatrix.sum(axis=1)

# using set_option to display the maximum size of the column to 10
pd.set_option('display.max_columns', 10)

# rounding the values upto two decimal places in the resulting confusion matrix and printing it.
CMNormalized = df_CMNormalized.round(2)
print(CMNormalized)

"""
converting into a numpy matrix inorder to access the diagonal elemention from the array
Method .as_matrix will be removed in a future version. Using .values instead to convert into a numpy matrix
numpy_matrix = df_confusion.as_matrix()
"""
numpy_matrix = CMNormalized.values

#access the diagonal elements from the numpy array
diagonalValue = numpy_matrix.diagonal(0)

# calculate the percentage value for finding the accuracy of the model on the test dataset and printing it.
sum=np.sum(diagonalValue)
accuracyPercentage = (sum*10)
print("\nAccuracy Percentage: ", accuracyPercentage)

#Printing predicted class label for each test image with serial number and correct class label
print('\nSNO \t Given label \t Predicted label')
print('----\t-------------\t-----------------')
for i in range(1,1001):
  print(i, '\t\t',testLabelList[i-1], '\t\t', confusionMatrix[i-1])



--------------- Confusion Matrix: ----------------

Predicted     0     1     2     3     4     5     6     7     8     9
Given                                                                
0          0.98  0.00  0.00  0.00  0.00  0.00  0.00  0.00  0.02  0.00
1          0.00  0.77  0.01  0.03  0.00  0.00  0.01  0.00  0.19  0.00
2          0.07  0.00  0.81  0.04  0.01  0.00  0.07  0.00  0.03  0.00
3          0.01  0.00  0.01  0.85  0.00  0.00  0.01  0.04  0.03  0.05
4          0.00  0.00  0.02  0.00  0.79  0.00  0.05  0.00  0.08  0.08
5          0.16  0.00  0.01  0.26  0.05  0.03  0.01  0.01  0.35  0.05
6          0.07  0.00  0.08  0.00  0.01  0.00  0.77  0.00  0.06  0.00
7          0.03  0.01  0.03  0.01  0.04  0.00  0.00  0.74  0.10  0.06
8          0.03  0.00  0.03  0.16  0.03  0.00  0.00  0.03  0.72  0.01
9          0.02  0.00  0.01  0.04  0.17  0.00  0.00  0.05  0.03  0.67

Accuracy Percentage:  71.3

SNO 	 Given label 	 Predicted label
----	-------------	-----------------
1 		 9